In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import string
import re

In [2]:
#read data for local machine
curr_dir= os.getcwd()
train = pd.read_csv(curr_dir+"//Dataset/train.tsv", sep='\t')
test = pd.read_csv(curr_dir+"//Dataset/test.tsv",sep='\t')

In [3]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [4]:
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [5]:
def cat_split(row):
    try:
        text = row
        txt1, txt2, txt3 = text.split('/')
        return txt1, txt2, txt3
    except:
        return np.nan, np.nan, np.nan

train["cat_1"], train["cat_2"], train["cat_3"] = zip(*train.category_name.apply(lambda val: cat_split(val)))
test["cat_1"], test["cat_2"], test["cat_3"] = zip(*test.category_name.apply(lambda val: cat_split(val)))


In [6]:
train.drop(['category_name'], axis=1, inplace = True)
test.drop(['category_name'], axis=1, inplace = True)

In [17]:
train.shape

(1482535, 10)

In [21]:
test.shape

(693359, 9)

In [7]:
train_with_brand_name= train.dropna(subset=['brand_name']) 
train_without_brand_name = train[train["brand_name"].isnull()]

In [19]:
test_with_brand_name= test.dropna(subset=['brand_name']) 
test_without_brand_name = test[test["brand_name"].isnull()]

In [8]:
train_without_brand_name.shape

(632682, 10)

In [9]:
train_with_brand_name.shape

(849853, 10)

In [22]:
test_without_brand_name.shape

(295525, 9)

In [23]:
test_with_brand_name.shape

(397834, 9)

In [24]:
train_with_brand_name['brand_name'].describe()

count     849853
unique      4809
top         PINK
freq       54088
Name: brand_name, dtype: object

In [26]:
brand_list_train= list(train_with_brand_name['brand_name'].unique())

In [29]:
len(brand_list_train)

4807

In [30]:
brand_list_test= list(test_with_brand_name['brand_name'].unique())
len(brand_list_test)

In [34]:
brand_list = list(set(brand_list_train+brand_list_test))
len(brand_list)

5289

In [63]:
brand_list.remove('X')
brand_list.remove('M')
brand_list = [w.replace('A|X Armani Exchange', 'Armani Exchange') for w in brand_list_train]

In [38]:
train_without_brand_name.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
0,0,MLB Cincinnati Reds T Shirt Size XL,3,NaN,10.0,1,No description yet,Men,Tops,T-shirts
3,3,Leather Horse Statues,1,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces
5,5,Bundled items requested for Ruie,3,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma...",Women,Other,Other
9,9,Porcelain clown doll checker pants VTG,3,NaN,8.0,0,I realized his pants are on backwards after th...,Vintage & Collectibles,Collectibles,Doll


In [39]:
train_with_brand_name.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
6,6,Acacia pacific tides santorini top,3,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,Women,Swimwear,Two-Piece
7,7,Girls cheer and tumbling bundle of 7,3,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,Sports & Outdoors,Apparel,Girls
8,8,Girls Nike Pro shorts,3,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,Sports & Outdoors,Apparel,Girls


In [ ]:
'''
regex = re.compile('[%s]' % re.escape(string.punctuation))
for name1,train_id1 in zip(train_without_brand_name['name'], train_without_brand_name['train_id']):
    name2= regex.sub('', name1)
    for brand_n in brand_list:
        name3 = name2.split()
        for name4 in name3:
            if name4 == brand_n:
                print(brand_n)
                print(name2)
                print(train_id1)
                mask = train_without_brand_name.train_id == train_id1
                train_without_brand_name.loc[mask, 'brand_name'] = brand_n
                train_without_brand_name['brand_name']= brand_n
                
'''            
                
        

In [111]:
%%time
i=0
for brand_n in brand_list:
    reg = re.compile('\\b'+brand_n+'\\b')
    if i == 0:
        brand_1 = train_without_brand_name[train_without_brand_name['name'].str.contains(reg)==True]
        brand_1.loc[:,'brand_name'] = brand_n
        i=1
    else:
        brand_2 = train_without_brand_name[train_without_brand_name['name'].str.contains(reg)==True]
        brand_2.loc[:,'brand_name'] = brand_n
        brand_1 = brand_1.append(brand_2)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 1h 16min 59s


In [126]:
brand_1 = brand_1.drop_duplicates(subset='train_id', keep='last')
brand_1.shape

(150679, 10)

In [127]:
train_with_brand_name = train_with_brand_name.append(brand_1)
train_without_brand_name = train_without_brand_name[~train_without_brand_name.index.isin(brand_1.index)]
train_with_added_brand_name = train_with_brand_name.append(train_without_brand_name)

In [168]:
train_with_added_brand_name.shape

(1482535, 10)

In [167]:
train_with_added_brand_name['brand_name'].isnull().sum()

482003

In [166]:
train_with_added_brand_name.to_csv(curr_dir+"//DataSet/train_with_brand_from_name.csv",index=False)

In [169]:
%%time
i=0
for brand_n in brand_list:
    reg = re.compile('\\b'+brand_n+'\\b')
    if i == 0:
        brand_1 = test_without_brand_name[test_without_brand_name['name'].str.contains(reg)==True]
        brand_1.loc[:,'brand_name'] = brand_n
        i=1
    else:
        brand_2 = test_without_brand_name[test_without_brand_name['name'].str.contains(reg)==True]
        brand_2.loc[:,'brand_name'] = brand_n
        brand_1 = brand_1.append(brand_2)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 35min 23s


In [171]:
brand_1 = brand_1.drop_duplicates(subset='test_id', keep='last')
brand_1.shape

(70353, 9)

In [172]:
test_with_brand_name = test_with_brand_name.append(brand_1)
test_without_brand_name = test_without_brand_name[~test_without_brand_name.index.isin(brand_1.index)]
test_with_added_brand_name = test_with_brand_name.append(test_without_brand_name)

In [173]:
test_with_added_brand_name.shape

(693359, 9)

In [174]:
test_with_added_brand_name['brand_name'].isnull().sum()

225172

In [175]:
test_with_added_brand_name.to_csv(curr_dir+"//DataSet/test_with_brand_from_name.csv",index=False)

In [ ]:
'''
This cell will take 12HOURS to run on entire data set(nearly 1million rows), when 
divided into 10 parts each having 100000 rows and run parallely on different kernels on 
a system having 32GB RAM, 1TB Hard Drive and i7 processor, 4Cores 

'''

'''
for name1,train_id1 in zip(train_without_brand_name['name'], train_without_brand_name['train_id']):
    for brand_n in brand_list:
        a=0
        match = re.search('\\b'+brand_n+'\\b', name1)
        try:
            match.group(0)
            a = 0
        except:
            a = 1
        if a == 0:
            print(brand_n)
            print(name1)
            print(train_id1)
            mask = train_without_brand_name.train_id == train_id1
            train_without_brand_name.loc[mask, 'brand_name'] = brand_n

train_without_brand_name.to_csv(curr_dir+"//Dataset/train_cat.csv")
'''

In [ ]:
'''
If we have run the above cell, then we need to run this code to get all files and combine them again

x_train1 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set1.csv", encoding = 'ISO-8859-1')
x_train2 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set2.csv", encoding = 'ISO-8859-1')
x_train3 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set3.csv", encoding = 'ISO-8859-1')
x_train4 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set4.csv", encoding = 'ISO-8859-1')
x_train5 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set5.csv", encoding = 'ISO-8859-1')
x_train6 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set6.csv", encoding = 'ISO-8859-1')
x_train7 = pd.read_csv(curr_dir+"//Price Suggestion/xtrain_set7.csv", encoding = 'ISO-8859-1')
x_test1 = pd.read_csv(curr_dir+"//Price Suggestion/xtest_set1.csv", encoding = 'ISO-8859-1')
x_test2 = pd.read_csv(curr_dir+"//Price Suggestion/xtest_set2.csv", encoding = 'ISO-8859-1')
x_test3 = pd.read_csv(curr_dir+"//Price Suggestion/xtest_set3.csv", encoding = 'ISO-8859-1')


x_train1.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_train2.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_train3.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_train4.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_train5.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_train6.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_train7.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_test1.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_test2.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)
x_test3.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace = True)

x_test = pd.concat([x_test1,x_test2,x_test3])
x_train = pd.concat([x_train1,x_train2,x_train3,x_train4,x_train5,x_train6,x_train7])
x_train.to_csv(curr_dir+"//Price Suggestion/train_null.csv",index=False)
x_test.to_csv(curr_dir+"//Price Suggestion/test_null.csv",index=False)
'''